# Install packages
a) press ] from Julia REPL; add "XX";
b) or run `import Pkg; Pkg.add("XX")` in REPL or the notebook as below.
https://docs.julialang.org/en/v1/stdlib/Pkg/

In [60]:
#import Pkg; 
#Pkg.add("FASTX", "Glob", "CodecZlib", "Statistics", "DataFrames", "CSV", "Plots")
using FASTX
using Glob
using CodecZlib
using Statistics
using DataFrames
using CSV
using Plots
plotlyjs() # some warnings about WebIO relevant to interative visualization

Plots.PlotlyJSBackend()

# Read and count read length

In [61]:
#put the code in function to save execution time
function read(path::String)
    readLengthStat = []
    fastqStatNames = ["sample", "median", "minimum", "maxmum", "mean", "std", "read_number", "quantile_0.1", "quantile_0.5", "quantile_0.9", "variance"]
    #Note change the path to be that on your local computer.
    inFastqFiles = glob(path)
    for inFastqFile in inFastqFiles
        sample = split(basename(inFastqFile), "_R1")[1]
    
        #using FASTX package https://biojulia.net/FASTX.jl/latest/
        reader = FASTQ.Reader(GzipDecompressorStream(open(inFastqFile)))
        readlengths = Int64[]
        for record in reader
            push!(readlengths, length(FASTQ.sequence(record)))
        end
        close(reader)
    
        push!(readLengthStat, sample, median(readlengths), minimum(readlengths), maximum(readlengths), mean(readlengths), std(readlengths), length(readlengths), quantile(readlengths, 0.1), quantile(readlengths, 0.5), quantile(readlengths, 0.9), var(readlengths))
    end
    #return readLengthStat
    
    #Create a dataframe for read length statistics
    #Convert the 'readLengthStat' Vector to a x*y Matrix, and then create a dataframe
    readMatrix = permutedims(reshape(readLengthStat, :, length(inFastqFiles)))
    readDF_tmp = DataFrame(readMatrix, fastqStatNames)
    #println(readDF)
    return readDF_tmp
end

#@time timing the execution of the code
@time readDF = read("./fastqs-trimmed/*_R1_001.fastq.gz")

  6.333046 seconds (13.91 M allocations: 2.495 GiB, 4.41% gc time)


,sample,median,minimum,maxmum,mean,std
,Any,Any,Any,Any,Any,Any
1,200ng-frag6min-RD41-0-7X_S10_L001,151.0,10,151,144.791,14.8082
2,300ng-frag6min-RD41-0-7X_S9_L001,151.0,13,151,144.409,15.2198
3,400ng-frag6min-RD41-0-7X_S8_L001,151.0,10,151,145.662,13.9676
4,500ng-frag4min-RD38-0-8X_S3_L001,151.0,10,151,134.322,22.5556
5,500ng-frag6min-RD38-0-8X_S2_L001,133.0,10,151,127.719,25.1532
6,500ng-frag6min-RD41-0-7X_S7_L001,151.0,10,151,144.826,14.8746
7,500ng-frag8min-RD38-0-8X_S1_L001,130.5,10,151,126.308,25.4834
8,50ng-frag4min-RD38-0-8X_S6_L001,151.0,10,151,131.828,24.8132
9,50ng-frag6min-RD38-0-8X_S5_L001,140.0,10,151,131.564,24.5208


In [62]:
## test plotting
#plot(readDF[!, "mean"])
#scatter(readDF[!, "mean"])

#using StatsPlots
#gr(size=(400,300)) # Plots.GRBackend()
#@df readDF plot([:mean :median :std], colour = [:red :blue :green], legend = :right)
#@df readDF scatter([:mean :median :std], colour = [:red :blue :green], legend = :right)

## Extract insert size statistics from Picard output

In [63]:
function insert(path::String)
    insertStat = []
    insertStatNames = ["SAMPLE", "MEDIAN", "MIN", "MAX", "MEAN", "STD", "READ_PAIRS", "WIDTH_OF_10%", "WIDTH_OF_50%", "WIDTH_OF_90%"]
    inInsertSizeFiles = glob(path)
    for inInsertSizeFile in inInsertSizeFiles
        sample = split(basename(inInsertSizeFile), ".insert")[1]

        f = open(inInsertSizeFile, "r")
        lines = readlines(f)
        close(f)

        #turn to the 8th line
        values = split(lines[8])
        push!(insertStat, sample, values[1], values[4], values[5], values[6], values[7], values[8], values[10], values[14], values[18])
    end
    
    insertMatrix = permutedims(reshape(insertStat, :, length(inInsertSizeFiles)))
    insertDF = DataFrame(insertMatrix, insertStatNames)
    #println(InsertDF)
end

@time insertDF = insert("./aligned-deduplicated-sizemetrics/*.insert.size.metrics.txt")

  0.003267 seconds (31.63 k allocations: 1.146 MiB)


,SAMPLE,MEDIAN,MIN,MAX,MEAN,STD
,Any,Any,Any,Any,Any,Any
1,200ng-frag6min-RD41-0-7X_S10_L001,236,14,1550569,270.759043,206.360653
2,300ng-frag6min-RD41-0-7X_S9_L001,231,22,739471,266.983115,201.399677
3,400ng-frag6min-RD41-0-7X_S8_L001,255,31,556367,297.794222,238.390585
4,500ng-frag4min-RD38-0-8X_S3_L001,189,13,498256,215.778265,166.810932
5,500ng-frag6min-RD38-0-8X_S2_L001,165,13,633168,184.045024,133.427357
6,500ng-frag6min-RD41-0-7X_S7_L001,241,14,700648,279.326327,219.084753
7,500ng-frag8min-RD38-0-8X_S1_L001,160,12,1137847,178.254623,126.156443
8,50ng-frag4min-RD38-0-8X_S6_L001,174,12,602343,193.745637,136.512506
9,50ng-frag6min-RD38-0-8X_S5_L001,170,11,603124,188.198179,127.515731


# Merge two tables into a CSV file

In [64]:
df = hcat(readDF, insertDF[!, Not(names(insertDF, "SAMPLE"))])
#CSV.write("./read_length_insert_size_summary.csv", df)

,sample,median,minimum,maxmum,mean,std
,Any,Any,Any,Any,Any,Any
1,200ng-frag6min-RD41-0-7X_S10_L001,151.0,10,151,144.791,14.8082
2,300ng-frag6min-RD41-0-7X_S9_L001,151.0,13,151,144.409,15.2198
3,400ng-frag6min-RD41-0-7X_S8_L001,151.0,10,151,145.662,13.9676
4,500ng-frag4min-RD38-0-8X_S3_L001,151.0,10,151,134.322,22.5556
5,500ng-frag6min-RD38-0-8X_S2_L001,133.0,10,151,127.719,25.1532
6,500ng-frag6min-RD41-0-7X_S7_L001,151.0,10,151,144.826,14.8746
7,500ng-frag8min-RD38-0-8X_S1_L001,130.5,10,151,126.308,25.4834
8,50ng-frag4min-RD38-0-8X_S6_L001,151.0,10,151,131.828,24.8132
9,50ng-frag6min-RD38-0-8X_S5_L001,140.0,10,151,131.564,24.5208


# Data visualization

In [65]:
scatter(readDF[!, "mean"], xlabel="sample", ylabel="mean of read length", label=false, smooth=true)

In [66]:
# convert an object of type SubString{String} to an object of type Float64
insertDF[!, "MEAN"] = [parse(Float64,x) for x in insertDF[!, "MEAN"]]

scatter(insertDF[!, "MEAN"], xlabel="sample", ylabel="mean of insert size", label=false, smooth=true)

In [67]:
scatter(readDF[!, "mean"], insertDF[!, "MEAN"], xlabel="mean of read length", ylabel="mean of insert size", label=false, smooth=true, legend = :bottomright)